![image](https://github.com/nicolas-thiers/Mec-270-Lab/blob/main/Lab05/Images/logo-mecanica.png?raw=1)

# Laboratorio 5: Resolución de ecuaciones lineales

Este notebook repasa los **métodos numéricos directos e iterativos para encontrar la solución de sistemas de ecuaciones lineales**. Se incluye una breve revisión de la teoría y los algoritmos generales de cada método, seguida de ejercicios prácticos.

## 1. Repaso de la teoría

**Problema General:** Encontrar el vector $\vec{x}$ que satisface el **sistema de ecuaciones lineales**: $A\vec{x} = \vec{b}$

Aquí, $A$ es una matriz cuadrada de tamaño $n \times n$ y $\vec{b}$ es un vector conocido. La solución buscada es $\vec{x}$. Resolver el sistema implica encontrar el vector $\vec{x} = A^{-1}\vec{b}$. Sin embargo, calcular la inversa de la matriz $A$ tiene un costo computacional elevado.

### 1.1 Métodos Directos

La idea principal de los métodos directos es **transformar el sistema de ecuaciones lineales original $A\vec{x} = \vec{b}$ en un sistema equivalente que sea más fácil de resolver**. Esto se logra pre-multiplicando (operando por la izquierda) por una matriz $P$ no singular: $P \cdot A\vec{x} = P \cdot \vec{b}$ . Dado que $P$ es no singular, $P^{-1}$ existe, y si $\vec{z}$ es la solución del sistema transformado $M A \vec{z} = M \vec{b}$, entonces $\vec{z} = A^{-1} \vec{b} = \vec{x}$, demostrando que la solución no cambia.

**Transformaciones útiles** (pre-multiplicación, no modifican la solución):
*   **Permutación:** Reordenamiento de las filas del sistema de ecuaciones lineales. Se realiza mediante una matriz de permutación $P$.
*   **Escalamiento diagonal:** Multiplicación de cada fila por un escalar distinto de cero. Se realiza mediante una matriz diagonal $D$.

**Sistemas Equivalentes Fáciles de Resolver**:
*   **Diagonal:** La matriz asociada es diagonal.
*   **Tri-diagonal:** La matriz asociada solo tiene elementos en la diagonal principal y en las diagonales inmediatamente superior e inferior.
*   **Triangular superior:** La matriz asociada $U$ tiene ceros debajo de la diagonal principal. Un sistema $U\vec{x} = \vec{b}$ se resuelve eficientemente mediante **sustitución hacia atrás**. El algoritmo es:

```
  for j = n to 1:
    if u_jj == 0 -> stop
    x_j = b_j / u_jj
    for i = 1 to j-1:
      b_i = b_i - u_ij * x_j
    end
  end
```
*   **Triangular inferior:** La matriz asociada $L$ tiene ceros encima de la diagonal principal. Un sistema $L\vec{x} = \vec{b}$ se resuelve eficientemente mediante **sustitución hacia adelante**. El algoritmo es:

```
  for j = 1 to n:
    if l_jj == 0 -> stop
    x_j = b_j / l_jj
    for i = j+1 to n:
      b_i = b_i - l_ij * x_j
    end
  end
```

#### Eliminación Gaussiana y Factorización LU

El objetivo es transformar la matriz $A$ en una matriz triangular superior mediante operaciones que reemplacen elementos por ceros. Estas operaciones se basan en la combinación lineal de filas y pueden representarse mediante **matrices de eliminación elemental** $M_k$. Una matriz $M_k$ puede transformar un vector columna $\vec{a}$ en un vector con ceros a partir de la fila $k+1$.

La **Eliminación Gaussiana** es un método para resolver $A\vec{x} = \vec{b}$ transformando el sistema original en un sistema triangular utilizando matrices de eliminación elemental. Aplica secuencialmente matrices de eliminación elemental a la matriz $A$ utilizando los vectores columna como pivotes. La secuencia de operaciones es:

$$ A\vec{x} = \vec{b} \Rightarrow M_1 A\vec{x} = M_1 \vec{b} \Rightarrow M_2 M_1 A\vec{x} = M_2 M_1 \vec{b} \Rightarrow \dots \Rightarrow M_n M_{n-1} \dots M_1 A\vec{x} = M_n M_{n-1} \dots M_1 \vec{b} $$

Si definimos $M = M_n M_{n-1} \dots M_1$, entonces $M A = U$, donde $U$ es una matriz triangular superior, y el sistema transformado es $U\vec{x} = M\vec{b}$. Este sistema triangular se resuelve luego mediante sustitución hacia atrás.

La inversa de la matriz de transformación $M$ es $M^{-1} = L$, donde $L$ es una matriz triangular inferior con unos en la diagonal. De la relación $MA=U$, pre-multiplicando por $M^{-1}$ obtenemos $A = M^{-1} U = L U$. Esta es la **factorización LU** de la matriz $A$. Resolver $A\vec{x} = \vec{b}$ se convierte en resolver $L U \vec{x} = \vec{b}$. Esto se hace en dos pasos: 1) Resolver $L\vec{y} = \vec{b}$ para $\vec{y}$ (sustitución hacia adelante). 2) Resolver $U\vec{x} = \vec{y}$ para $\vec{x}$ (sustitución hacia atrás).

El algoritmo básico para la factorización LU mediante eliminación Gaussiana es:

```
  for k = 1 to n-1:
    if a_kk == 0 -> stop
    for i = k+1 to n:
      m_ik = a_ik / a_kk
    end
    for j = k+1 to n:
      for i = k+1 to n:
        a_ij = a_ij - m_ik * a_kj
      end
    end
  end
```

### 1.2 Métodos Iterativos

Resolver el sistema $A\vec{x} = \vec{b}$ calculando $A^{-1}$ es de elevado costo. Los métodos directos como la factorización LU son una alternativa. Sin embargo, para **sistemas grandes**, los costos de memoria y computacionales de los métodos directos pueden ser prohibitivos. En estos casos, o para **sistemas matriciales especiales**, es posible utilizar un **método iterativo que converge asintóticamente a la solución**.

Los métodos iterativos presentan una ventaja computacional para:
*   Matrices diagonales dominantes (DD) ($|a_{ii}| > \sum_{j \neq i} |a_{ij}|$).
*   Matrices simétricas y definidas positivas (SDP) ($A^T = A$ y $\vec{x}^T A \vec{x} > 0$ para todo $\vec{x} \neq \vec{0}$).
*   Matrices sparse (con un número reducido de elementos no nulos).
*   Matrices triangulares (superior o inferior).
*   Matrices bandwidth ($a_{ij}=0$ si $|i-j| > k$).
*   Cuando es fácil estimar una solución inicial.

La implementación de métodos numéricos para resolver EDPs transientes a menudo produce sistemas matriciales sparse y simétricos, donde la solución del tiempo pasado puede usarse como una buena aproximación inicial para el tiempo presente.

#### Iteración Estacionaria

El esquema iterativo más sencillo es el de iteración estacionaria:

$$ \vec{x}^{(k+1)} = T\vec{x}^{(k)} + \vec{c} $$

Donde la matriz $T$ y el vector $\vec{c}$ se eligen de forma que un punto fijo de la iteración ($\vec{x}^* = T\vec{x}^* + \vec{c}$) sea la solución del sistema $A\vec{x} = \vec{b}$ . Se denomina estacionario porque $T$ y $\vec{c}$ no dependen de la iteración $k$.

La matriz $T$ se construye separando la matriz $A$ en $A = M - N$, donde $M$ es una matriz no singular y fácil de invertir. Entonces, $T = M^{-1} N$ y $\vec{c} = M^{-1} \vec{b}$. La iteración se puede escribir como $M \vec{x}^{(k+1)} = N \vec{x}^{(k)} + \vec{b}$, lo que implica resolver un sistema lineal con matriz $M$ en cada iteración. La idea es que $M$ sea similar a $A$ pero fácil de invertir (e.g., diagonal, triangular inferior o superior).

La convergencia del esquema iterativo está garantizada si el radio espectral de la matriz Jacobiana asociada $J = \partial G / \partial \vec{x} = T = M^{-1}N$ cumple $\rho(J) < 1$. A menor radio espectral, mayor convergencia. Hay un balance entre la facilidad de invertir $M$ y el radio espectral. El caso extremo $M=A$ converge en una iteración, pero es computacionalmente inviable si $A$ es grande.

#### Método de Jacobi

Es un método de iteración estacionaria. Se escoge la matriz $M$ como la diagonal de $A$, $M = D = \text{diag}(A)$, y $-N$ como el resto de la matriz, $N = -(L + U)$, donde $L$ y $U$ son las partes triangular inferior y superior de $A$ respectivamente. Si los elementos diagonales $a_{ii}$ son no nulos, $D$ es invertible.

El esquema iterativo es:

$$ \vec{x}^{(k+1)} = D^{-1} (L + U) \vec{x}^{(k)} + D^{-1} \vec{b} $$

La convergencia está garantizada si la matriz $A$ es diagonal dominante o simétrica definida positiva.

El algoritmo del método de Jacobi es:

```
  x(0) = ... (aproximación inicial)
  res = ||A x(0) - b||_p
  k = 0
  while (res > tol or k < kmax) do
    for i = 1, 2, ..., n do
      x(k+1)_i = (b_i - sum_{j=1; j!=i}^{n} a_ij * x(k)_j) / a_ii
    end for
    res = ||A x(k+1) - b||_p
    k += 1
  end for
```

#### Método de Gauss-Seidel

Una limitación del método de Jacobi es su lenta convergencia, en parte porque no utiliza la información de la iteración actual tan pronto como está disponible. El método de Gauss-Seidel es una modificación que utiliza la información de la iteración actual para calcular la nueva aproximación.

Se escoge la matriz $M$ como la suma de la diagonal y la parte triangular inferior de $A$, $M = L + D$, y $-N$ como la parte triangular superior, $N = -U$.

El esquema iterativo es:

$$ \vec{x}^{(k+1)} = (L + D)^{-1} (-U \vec{x}^{(k)} + \vec{b}) $$

Partiendo de $\vec{x}^{(0)}$, se aproxima la solución resolviendo un sistema lineal con matriz triangular inferior $(L+D)$. En forma de componentes, la fórmula es:

$$ x_i^{(k+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)}}{a_{ii}} $$

Observe que para calcular $x_i^{(k+1)}$ se utilizan los valores de $x_j^{(k+1)}$ que ya han sido actualizados en la iteración actual para $j < i$, y los valores de $x_j^{(k)}$ de la iteración anterior para $j > i$.

La convergencia está garantizada si la matriz $A$ es simétrica definida positiva.

El algoritmo del método de Gauss-Seidel es:

```
  x = ... (aproximación inicial)
  res = ||A x - b||_p
  k = 0
  while (res > tol or k < kmax) do
    for i = 1, 2, ..., n do
      x_i = (b_i - sum_{j=1}^{i-1} a_ij * x_j - sum_{j=i+1}^{n} a_ij * x_j) / a_ii
    end for
    res = ||A x - b||_p
    k += 1
  end for
```

#### Método del Gradiente Conjugado (CG - Conjugate Gradient)

El método del gradiente conjugado es un método iterativo (aunque por construcción es directo) basado en procesos de optimización. Requiere que la matriz $A$ sea **simétrica definida positiva**.

La solución del sistema $A\vec{x} = \vec{b}$ minimiza la función cuadrática $\psi(\vec{x}) = \frac{1}{2} \vec{x}^T A \vec{x} - \vec{x}^T \vec{b}$, ya que su gradiente $\nabla\psi(\vec{x}) = A\vec{x} - \vec{b}$ es cero en la solución.

La idea es moverse desde un punto inicial $\vec{x}^{(0)}$ hacia el mínimo mediante pasos discretos en una dirección $\vec{s}^{(k)}$ una distancia $\alpha$:

$$ \vec{x}^{(k+1)} = \vec{x}^{(k)} + \alpha \vec{s}^{(k)} $$

La distancia $\alpha$ óptima para cualquier dirección de búsqueda $\vec{s}^{(k)}$ es aquella que hace que el nuevo vector residuo $\vec{r}^{(k+1)} = \vec{b} - A\vec{x}^{(k+1)}$ sea perpendicular a la dirección de búsqueda $\vec{s}^{(k)}$: $\vec{r}^{(k+1)T} \vec{s}^{(k)} = 0$. El residuo se actualiza como $\vec{r}^{(k+1)} = \vec{r}^{(k)} - \alpha A\vec{s}^{(k)}$. El paso óptimo $\alpha$ se calcula como:

$$ \alpha = \frac{\vec{r}^{(k)T} \vec{s}^{(k)}}{\vec{s}^{(k)T} A \vec{s}^{(k)}} $$

Una elección simple para la dirección de búsqueda sería el negativo del gradiente, $\vec{s}^{(k)} = -\nabla\psi(\vec{x}^{(k)}) = \vec{r}^{(k)}$ (método del descenso del gradiente). Sin embargo, esto puede llevar a convergencia sub-óptima. El método del gradiente conjugado utiliza direcciones de búsqueda que son **ortogonales respecto a la matriz A (A-conjugadas)**: $\vec{s}^{(k+1)T} A \vec{s}^{(k)} = 0$.

El algoritmo del método de gradiente conjugado es:

```
  x(0) = ... (aproximación inicial)
  r(0) = b - A * x(0)
  s(0) = r(0)
  k = 0
  while (||r(k)||_p > tol or k < kmax) do
    alpha = (r(k)' * r(k)) / (s(k)' * A * s(k))
    x(k+1) = x(k) + alpha * s(k)
    r(k+1) = r(k) - alpha * A * s(k)
    beta = (r(k+1)' * r(k+1)) / (r(k)' * r(k))
    s(k+1) = r(k+1) + beta * s(k)
    k += 1
  end while
```

In [1]:
### def SolveEliminacionGauss(A_, b_):
###     # Implementación de eliminacion Gaussiana
###     return xHat
import numpy as np

def SolveEliminacionGauss(A_, b_):
    n = len(b_)
    A = A_.copy().astype(float)
    b = b_.copy().astype(float)

    # Eliminación hacia adelante
    for k in range(n-1):
        max_row = np.argmax(np.abs(A[k:, k])) + k
        if max_row != k:
            A[[k, max_row]] = A[[max_row, k]]
            b[[k, max_row]] = b[[max_row, k]]

        if A[k, k] == 0:
            raise ValueError("Matriz singular detectada durante la eliminación")

        for i in range(k+1, n):
            factor = A[i, k] / A[k, k]
            A[i, k:] -= factor * A[k, k:]
            b[i] -= factor * b[k]

    # Sustitución hacia atrás
    xHat = np.zeros(n)
    for i in range(n-1, -1, -1):
        if A[i, i] == 0:
            raise ValueError("Matriz singular detectada durante la sustitución")
        xHat[i] = (b[i] - np.dot(A[i, i+1:], xHat[i+1:])) / A[i, i]

    return xHat

In [22]:
### def SolveJacobi(A_, b_, x0_, tol_=1e-6, kmax_=1000):
###     # Implementación del Método de Jacobi
###     return xHat, k
def SolveJacobi(A_, b_, x0_, tol_=1e-6, kmax_=1000):
    n = len(b_)
    xHat = x0_.copy().astype(float)
    x_new = np.zeros_like(xHat)
    k = 0
    res = np.linalg.norm(A_ @ xHat - b_, 2)

    while res > tol_ and k < kmax_:
        for i in range(n):
            sigma = np.dot(A_[i, :i], xHat[:i]) + np.dot(A_[i, i+1:], xHat[i+1:])
            x_new[i] = (b_[i] - sigma) / A_[i, i]

        xHat = x_new.copy()
        res = np.linalg.norm(A_ @ xHat - b_, 2)
        k += 1

    return xHat, k

In [24]:
### def SolveGaussSeidel(A_, b_, x0_, tol_=1e-6, kmax_=1000):
###     # Implementación del Método de Gauss-Seidel
###     return xHat, k
def SolveGaussSeidel(A_, b_, x0_, tol_=1e-6, kmax_=1000):
    n = len(b_)
    xHat = x0_.copy().astype(float)
    k = 0
    res = np.linalg.norm(A_ @ xHat - b_, 2)

    while res > tol_ and k < kmax_:
        for i in range(n):
            sigma = np.dot(A_[i, :i], xHat[:i]) + np.dot(A_[i, i+1:], xHat[i+1:])
            xHat[i] = (b_[i] - sigma) / A_[i, i]

        res = np.linalg.norm(A_ @ xHat - b_, 2)
        k += 1

    return xHat, k

In [26]:
### def SolveCG(A_, b_, x0_, tol_=1e-6, kmax_=None):
###     # Implementación del Método de Gradiente Conjugado
###     # Requiere que A sea Simétrica Definida Positiva
###     return xHat, k
def SolveCG(A_, b_, x0_, tol_=1e-6, kmax_=None):
    n = len(b_)
    if kmax_ is None:
        kmax_ = n

    xHat = x0_.copy().astype(float)
    r = b_ - A_ @ xHat
    s = r.copy()
    k = 0
    res = np.linalg.norm(r, 2)

    while res > tol_ and k < kmax_:
        alpha = np.dot(r, r) / np.dot(s, A_ @ s)
        xHat = xHat + alpha * s
        rn = r - alpha * (A_ @ s)
        beta = np.dot(rn, rn) / np.dot(r, r)
        s = rn + beta * s
        r = rn
        res = np.linalg.norm(r, 2)
        k += 1

    return xHat, k

## 2. Ejercicios propuestos

### Distribución de Temperaturas en una Barra (Conducción 1D en Estado Estacionario)

Considere una barra metálica de longitud $L$ con área de sección transversal constante $A_c$, conductividad térmica $k$. La barra está aislada en sus superficies laterales y sufre convección con el ambiente a temperatura $T_\infty$ con un coeficiente de convección $h$. Los extremos de la barra se mantienen a temperaturas fijas $T_0$ en $x=0$ y $T_L$ en $x=L$.

![image](https://github.com/nicolas-thiers/Mec-270-Lab/blob/main/Lab05/Images/Esquema-1.svg?raw=1)

La ecuación de gobierno para la distribución de temperatura $T(x)$ en estado estacionario es:

$$ \frac{d}{dx} \left( k \frac{dT}{dx} \right) - \frac{h P}{A_c} (T - T_\infty) = 0 $$

donde $P$ es el perímetro de la sección transversal.

Utilizaremos el método de diferencias finitas para discretizar esta ecuación en $N$ nodos espaciados uniformemente a lo largo de la barra ($x_i = i \Delta x$, donde $\Delta x = L/(N-1)$). Aproximando las derivadas segundas:

$$ \frac{d^2T}{dx^2} \approx \frac{T_{i-1} - 2T_i + T_{i+1}}{(\Delta x)^2} $$

Asumiendo conductividad $k$ constante y definiendo $m^2 = h P / (k A_c)$, la ecuación discretizada en el nodo $i$ (para $i=1, \dots, N-2$) es:

$$ k \frac{T_{i-1} - 2T_i + T_{i+1}}{(\Delta x)^2} - m^2 k (T_i - T_\infty) = 0 $$

Reordenando, obtenemos una ecuación lineal para cada nodo interior $i$:

$$ T_{i-1} - (2 + m^2 (\Delta x)^2) T_i + T_{i+1} = -m^2 (\Delta x)^2 T_\infty $$

Para los nodos en los extremos ($i=0$ e $i=N-1$), las condiciones de frontera son simplemente $T_0$ y $T_{N-1} = T_L$.

Esto nos lleva a un sistema de $N$ ecuaciones lineales con $N$ incógnitas ($T_0, T_1, \dots, T_{N-1}$) de la forma $A\vec{T} = \vec{b}$. Para $N=5$ nodos, el sistema se ve así:

$$i=0: T_0 = T_{\text{frontera},0}$$

$$i=1: T_0 - (2 + m^2 (\Delta x)^2) T_1 + T_2 = -m^2 (\Delta x)^2 T_\infty$$

$$i=2: T_1 - (2 + m^2 (\Delta x)^2) T_2 + T_3 = -m^2 (\Delta x)^2 T_\infty$$

$$i=3: T_2 - (2 + m^2 (\Delta x)^2) T_3 + T_4 = -m^2 (\Delta x)^2 T_\infty$$

$$i=4: T_4 = T_{\text{frontera},L}$$

Sustituyendo las condiciones de frontera en las ecuaciones para $i=1$ y $i=N-2=3$:

$$i=1: - (2 + m^2 (\Delta x)^2) T_1 + T_2 = -m^2 (\Delta x)^2 T_\infty - T_{\text{frontera},0}$$

$$i=3: T_2 - (2 + m^2 (\Delta x)^2) T_3 = -m^2 (\Delta x)^2 T_\infty - T_{\text{frontera},L}$$

El sistema para los $N-2$ nodos interiores ($T_1, T_2, T_3$) es:

$$\begin{bmatrix} -(2 + m^2 (\Delta x)^2) & 1 & 0 \\ 1 & -(2 + m^2 (\Delta x)^2) & 1 \\ 0 & 1 & -(2 + m^2 (\Delta x)^2) \end{bmatrix} \begin{bmatrix} T_1 \\ T_2 \\ T_3 \end{bmatrix} = \begin{bmatrix} -m^2 (\Delta x)^2 T_\infty - T_{\text{frontera},0} \\ -m^2 (\Delta x)^2 T_\infty \\ -m^2 (\Delta x)^2 T_\infty - T_{\text{frontera},L} \end{bmatrix}$$

Defina $C = 2 + m^2 (\Delta x)^2$ y $R = -m^2 (\Delta x)^2 T_\infty$. El sistema es:

$$ \begin{bmatrix} -C & 1 & 0 \\ 1 & -C & 1 \\ 0 & 1 & -C \end{bmatrix} \begin{bmatrix} T_1 \\ T_2 \\ T_3 \end{bmatrix} = \begin{bmatrix} R - T_{\text{frontera},0} \\ R \\ R - T_{\text{frontera},L} \end{bmatrix} $$

Este es un sistema de 3x3 para los nodos interiores. La matriz del sistema es **tri-diagonal**, lo que la hace un tipo de matriz especial. Para un número mayor de nodos $N$, la matriz del sistema para los $N-2$ nodos interiores será de tamaño $(N-2) \times (N-2)$ y también tri-diagonal.

**Datos:**

1.  Longitud de la barra $L = 1$ m
2.  Parámetro $m^2 = 10$ m$^{-2}$
3.  Temperatura ambiente $T_\infty = 20$ °C
4.  Temperatura en $x=0$, $T_0 = 100$ °C
5.  Temperatura en $x=L$, $T_L = 50$ °C
6.  Número de nodos $N=10$
7.  Tolerancia para métodos iterativos $tol = 10^{-6}$
8.  Máximo de iteraciones para métodos iterativos $k_{max} = 1000$

**Tareas:**

1.  Calcule el tamaño de la matriz del sistema lineal para los nodos interiores ($N-2$) y el valor de $\Delta x$.
2.  Construya la matriz $A$ y el vector $\vec{b}$ para el sistema de $N-2$ ecuaciones lineales para los nodos interiores.
3.  Resuelva el sistema utilizando un **método directo**. Reporte las temperaturas $T_1, T_2, \dots, T_{N-2}$.
4.  Resuelva el sistema utilizando los **métodos iterativos** de Jacobi, Gauss-Seidel . Use un vector inicial de ceros (o todos unos). Reporte las temperaturas $T_1, \dots, T_{N-2}$ y el número de iteraciones necesarias para alcanzar la tolerancia $tol$. Calcule el residuo final $||A\hat{x} - \vec{b}||_p$.
5.  Resuelva el sistema utilizando el **método del Gradiente Conjugado**. Asegúrese de que la matriz $A$ sea simétrica definida positiva para aplicar este método. Reporte las temperaturas y el número de iteraciones. Compare la convergencia (respecto al número de iteraciones) con Jacobi y Gauss-Seidel.


In [27]:

#Manuel Casanova Casanova
# Parámetros del problema
L = 1.0          # Longitud de la barra (m)
m2 = 10.0        # Parámetro m^2 (m^-2)
T_inf = 20.0     # Temperatura ambiente (°C)
T0 = 100.0       # Temperatura en x=0 (°C)
TL = 50.0        # Temperatura en x=L (°C)
N = 10           # Número de nodos
tol = 1e-6       # Tolerancia para métodos iterativos
kmax = 1000      # Máximo de iteraciones

# 1. Calcular tamaño de la matriz y Δx
Delta_x = L / (N - 1)
nodos_interiores = N - 2
print(f"Tamaño de la matriz del sistema: {nodos_interiores}x{nodos_interiores}")
print(f"Delta_x = {Delta_x:.4f} m")

# 2. Construir la matriz A y el vector b
C = 2 + m2 * Delta_x**2
R = -m2 * Delta_x**2 * T_inf

# Inicializar matriz A (tri-diagonal)
A = np.zeros((nodos_interiores, nodos_interiores))
for i in range(nodos_interiores):
    A[i, i] = -C
    if i > 0:
        A[i, i-1] = 1
    if i < nodos_interiores - 1:
        A[i, i+1] = 1

# Construir vector b
b = np.full(nodos_interiores, R)
b[0] -= T0
b[-1] -= TL

print("\nMatriz A:")
print(A)
print("\nVector b:")
print(b)

# 3. Resolver usando método directo (Eliminación Gaussiana)
T_direct = SolveEliminacionGauss(A, b)
print("\nSolución por método directo (Eliminación Gaussiana):")
print(T_direct)

# 4. Resolver usando métodos iterativos
# Vector inicial (ceros o unos)
x0 = np.zeros(nodos_interiores)

# Método de Jacobi
T_jacobi, iter_jacobi = SolveJacobi(A, b, x0, tol, kmax)
residual_jacobi = np.linalg.norm(A @ T_jacobi - b, 2)
print(f"\nSolución por Jacobi ({iter_jacobi} iteraciones, residual={residual_jacobi:.2e}):")
print(T_jacobi)

# Método de Gauss-Seidel
T_gs, iter_gs = SolveGaussSeidel(A, b, x0, tol, kmax)
residual_gs = np.linalg.norm(A @ T_gs - b, 2)
print(f"\nSolución por Gauss-Seidel ({iter_gs} iteraciones, residual={residual_gs:.2e}):")
print(T_gs)

# 5. Resolver usando Gradiente Conjugado
# Verificar si la matriz es simétrica definida positiva
if np.allclose(A, A.T):
    print("\nLa matriz A es simétrica")
    # Verificar si es definida positiva (todos los autovalores > 0)
    eigenvalues = np.linalg.eigvals(A)
    if np.all(eigenvalues > 0):
        print("La matriz A es definida positiva")

        T_cg, iter_cg = SolveCG(A, b, x0, tol, kmax)
        residual_cg = np.linalg.norm(A @ T_cg - b, 2)
        print(f"\nSolución por Gradiente Conjugado ({iter_cg} iteraciones, residual={residual_cg:.2e}):")
        print(T_cg)
    else:
        print("La matriz A no es definida positiva (no se puede aplicar CG)")
else:
    print("\nLa matriz A no es simétrica (no se puede aplicar CG)")

# Comparación de resultados
print("\nComparación de resultados:")
print(f"Eliminación Gaussiana: {T_direct}")
print(f"Jacobi:              {T_jacobi}")
print(f"Gauss-Seidel:        {T_gs}")
if 'T_cg' in locals():
    print(f"Gradiente Conjugado: {T_cg}")

# Comparación de convergencia
print("\nComparación de convergencia (número de iteraciones):")
print(f"Jacobi: {iter_jacobi}")
print(f"Gauss-Seidel: {iter_gs}")
if 'iter_cg' in locals():
    print(f"Gradiente Conjugado: {iter_cg}")




Tamaño de la matriz del sistema: 8x8
Delta_x = 0.1111 m

Matriz A:
[[-2.12345679  1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.         -2.12345679  1.          0.          0.          0.
   0.          0.        ]
 [ 0.          1.         -2.12345679  1.          0.          0.
   0.          0.        ]
 [ 0.          0.          1.         -2.12345679  1.          0.
   0.          0.        ]
 [ 0.          0.          0.          1.         -2.12345679  1.
   0.          0.        ]
 [ 0.          0.          0.          0.          1.         -2.12345679
   1.          0.        ]
 [ 0.          0.          0.          0.          0.          1.
  -2.12345679  1.        ]
 [ 0.          0.          0.          0.          0.          0.
   1.         -2.12345679]]

Vector b:
[-102.4691358   -2.4691358   -2.4691358   -2.4691358   -2.4691358
   -2.4691358   -2.4691358  -52.4691358]

Solución por método directo (Eliminación Gaussiana):
[77.2152